In [18]:
import networkx as nx
import pandas as pd
import numpy as np
import os
if "DATASET_DIR" not in os.environ:
    os.environ["DATASET_DIR"] =  "/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/jupyter/datasets"

In [53]:
def get_out_degree(data):
    """ data Array of size D, 2. Where d is number of edges and src, dest """
    v = {}
    for src, dest in data:
        v[src] = v.get(dest,0) + 1
    return sum(v.values()) / len(v) if len(v) > 0 else 0

def get_running_out_degree(data):
    l = []
    for i in range(1000, data.shape[0], 1000):
        if i % 10000 == 0:
            print(i)
        l.append(get_out_degree(data[[0,1]][:i].values))
    return sum(l) / len(l) if len(l) > 0 else 0

# Reddit Hyperlinks

In [51]:
body = pd.read_csv(os.path.join(os.getenv("DATASET_DIR"), "RedditHyperlinks/soc-redditHyperlinks-body.tsv"), header=None, delimiter="\t", usecols=[0,1,2])

In [54]:
avg_out_degree = get_running_out_degree(body)
print(avg_out_degree)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
5.275899257287914


# StackOverflow

In [5]:
data = pd.read_csv("../datasets/StackOverflow/sx-stackoverflow.txt", header=None, delimiter=" ")
data.to_csv("../datasets/StackOverflow/sx-stackoverflow.tsv", index=False, header=False)